# Key Findings

1. As of 2024, the metros with the highest number of data science employees are as follows: 
    * New York-Newark-Jersey City, NY-NJ
    * Los Angeles-Long Beach-Anaheim, CA
    * San Francisco-Oakland-Fremont, CA
    * Dallas-Fort Worth-Arlington, TX
    * Washington-Arlington-Alexandria, DC-VA-MD-WV
    


# Prep

Data Sources: 
* OEWS Tables - https://www.bls.gov/oes/tables.htm 
* Shapefiles - https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2024&layergroup=Core+Based+Statistical+Areas

In [15]:
#!pip install geopandas shapely plotly
#!pip install thefuzz
#!pip install --upgrade nbformat


In [16]:
import pandas as pd  ### Basic data manipulations
import matplotlib.pyplot as plt
import os
import geopandas as gpd ### using shapefiles

import plotly.express as px ### choropleth map
import plotly.graph_objects as go

import json ### to read the converted shapefiles
from thefuzz import process ### for merging on MSA with slightly different labeling patterns
import numpy as np  ### dealing with missing data and other calcs

import warnings
warnings.filterwarnings("ignore")  ### warnings sometimes show your computer, so I want to surpress if sending to github


# Top Industries in 2024

In [17]:
industry_4digit_df = pd.read_excel('Data\\nat4d_M2024_dl.xlsx', sheet_name="Nat4d_M2024_dl") ### read in national employment by industry
industry_3digit_df = pd.read_excel('Data\\nat3d_M2024_dl.xlsx', sheet_name="nat3d_M2024_dl")

In [18]:
def create_industry_rank(df): 
    create_df = df[df['OCC_CODE']=='15-2051'] ### # Filter data data science jobs 
    create_df = create_df[create_df['OWN_CODE']==5] ### filter to private industry
    create_df["TOT_EMP"] = pd.to_numeric(create_df["TOT_EMP"], errors="coerce") ### Convert to numeric - some * for missing data
    create_df = create_df.sort_values(by="TOT_EMP", ascending=False) ### Sort by employment
    return(create_df)

In [19]:
nation_datascience4 = create_industry_rank(industry_4digit_df)
nation_datascience3 = create_industry_rank(industry_3digit_df)

# MAP MSAs

There are subtle differences in some of the naming conventions for the MSAs in the census and the Occupational Employment and Wage Statistics (OEWS) data so we have to do some matching to make a map between the two data sources. 


## Load Data

In [20]:
# Load the shapefile
shapefile_path = "CensusShapeFiles/tl_2024_us_cbsa.shp"
gdf = gpd.read_file(shapefile_path)
census_locale =gdf[['NAME']]

In [21]:
# Load OEWS Data -- this step will take awhile to read-in the full df
Employment = pd.read_excel('Data\\MSA_M2024_dl.xlsx', sheet_name="MSA_M2024_dl")  ### read in the data
OEWS_locale = Employment[['AREA_TITLE']].drop_duplicates()

## Merge data

Merge 1 --  Exact matches

In [22]:
# Keeping exact matches (based on the data in OEWS df)
exact_match = pd.merge(census_locale, OEWS_locale, left_on='NAME', right_on='AREA_TITLE', how='inner')

In [23]:
# Checking un-matched geographies
anti_join = pd.merge(census_locale, OEWS_locale, left_on='NAME', right_on='AREA_TITLE', how='right')
anti_join = anti_join[anti_join['NAME'].isna()] ### keep only NA values to find what BLS values did not have a geo match
anti_join = anti_join.dropna(axis=1, how='all') ### dropping cols filled with NAs to try a fuzzy match on remaining missing locations


Merge 2 -- Fuzzy Matches

In [24]:
# Function to find best match
def fuzzy_merge(census_locale, anti_join, left_col, right_col, threshold=88):
    matches = census_locale[left_col].apply(lambda x: process.extractOne(x, anti_join[right_col], score_cutoff=threshold))
    census_locale['Best_Match'] = [match[0] if match else None for match in matches]
    return pd.merge(census_locale, anti_join, left_on='Best_Match', right_on=right_col, how='right').drop(columns=['Best_Match'])

# Perform Fuzzy Merge
fuzzy_df = fuzzy_merge(census_locale, anti_join, 'NAME', 'AREA_TITLE')
fuzzy_df.head() ### manually check that the fuzzy produced logical matches


,NAME,AREA_TITLE
0,"Mayagüez, PR","Mayaguez, PR"
1,"San Juan-Bayamón-Caguas, PR","San Juan-Bayamon-Caguas, PR"


In [25]:
# Combine MSA Maps into a final map
MSA_MAP = pd.concat([exact_match, fuzzy_df], ignore_index=True)

# Where the Data Jobs Are: Mapping Data Science Hotspots in the U.S.

## Prep data tables

For this analysis, I am going to explore which metropolitan areas had the highest number of data science jobs across the United States. 

Data Sources: 
* Occupational Employment and Wage Statistics (OEWS) Tables https://www.bls.gov/oes/tables.htm 
* Shapefiles: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2024&layergroup=Core+Based+Statistical+Areas

First, let's read in the BLS employment by occupation data and filter to just data science jobs

In [26]:
def create_MSAEmploy(df): 
    #  OEWS Data
    create_employ = df[df['OCC_CODE']=='15-2051'] ### Filter to just data science occupations
    create_employ ["TOT_EMP"] = pd.to_numeric(create_employ["TOT_EMP"], errors="coerce") ### Convert to numeric 
    create_employ = create_employ.dropna(subset=["TOT_EMP"])  ### Remove rows where TOT_EMP is not reported
    create_employ = create_employ.sort_values(by="TOT_EMP", ascending=False) ### Sort greatest to least
    create_employ["A_MEDIAN"] = pd.to_numeric(create_employ["A_MEDIAN"], errors="coerce") ### Convert to numeric 
    return(create_employ)

MSA_compare = create_MSAEmploy(Employment)

In [27]:
# Use the ID map to create a column to be able to merge the employment data with the shapefile 
MSA_employ = pd.merge(MSA_compare, MSA_MAP, left_on='AREA_TITLE', right_on='AREA_TITLE', how='inner')

Now let's read in the shape files from the census bureau.

In [28]:
# Extract MSA centroids
gdf["centroid"] = gdf.geometry.centroid
gdf["lon"] = gdf["centroid"].x
gdf["lat"] = gdf["centroid"].y


## Merge BLS data and shapefile into one df

In [29]:
# Combine the shapefile with the employment data
geo_employ = pd.merge(gdf, MSA_employ, left_on="NAME", right_on="NAME", how = 'right') ### want a right join because only want metros w/ employment data

In [41]:
# Choose a column to determine bubble size (replace 'POPULATION' with actual column)
if "TOT_EMP" in geo_employ.columns:
    geo_employ["Employment Count"] = geo_employ["TOT_EMP"]
else:
    geo_employ["Employment Count"] = 10  # Default size if no population data

# Create a DataFrame for Plotly
figure_df = geo_employ[["lon", "lat", "NAME", "Employment Count", "A_MEDIAN"]]  # Replace with actual column names

# Set up the mapbox layout
fig = px.scatter_mapbox(figure_df, 
                        lon="lon", 
                        lat="lat", 
                        size="Employment Count",  # Bubble size based on employment count
                        color="Employment Count",  # Color bubbles based on employment count
                        hover_name="NAME",  # Name to show on hover
                        size_max=35,  # Max bubble size
                        color_continuous_scale="Plasma",  # Color scale for employment count
                        title="Data Scientists <br><sup>Employment Count, by Metropolitan Area")

# Update the map style and layout
fig.update_layout(
    mapbox_style="carto-positron",  # You can choose other map styles like "open-street-map", "stamen-terrain", etc.
    mapbox_zoom=3,  # Zoom level for the map
    mapbox_center={"lat": 37.0902, "lon": -95.7129},  # Center map on the U.S., you can adjust this as needed
    showlegend=True  # Ensure the legend is shown
)


fig.update_traces(
    hovertemplate="<b>%{hovertext}</b><br>Employment Count: %{marker.size:,}<extra></extra>"
)

# Show the map
fig.show()

fig.write_html("interactive_map.html")

# Where are the fastest growing metros?


Note: When comparing changes in BLS OEWS numbers over time, it's important to ensure consistency in job definitions and estimation methodology.
* The Standard Occupational Classification (SOC) system was updated in 2018, which reclassified many occupations. Therefore, data prior to 2018 may not be comparable to more recent years.
* In 2021, the OEWS program adopted the MB3 estimation methodology. Only estimates from 2021 onward, which use the MB3 model, should be compared for accurate trend analysis.

## Clean historical dfs

In [31]:
# read-in other data
Employment2021 = pd.read_excel('Data\\MSA_M2021_dl.xlsx', sheet_name="MSA_M2021_dl")  ### read in the data
Employment2022 = pd.read_excel('Data\\MSA_M2022_dl.xlsx', sheet_name="MSA_M2022_dl")  ### read in the data
Employment2023 = pd.read_excel('Data\\MSA_M2023_dl.xlsx', sheet_name="MSA_M2023_dl")  ### read in the data

In [32]:
MSA_employ2021 = create_MSAEmploy(Employment2021) ### Function defined in mapping section
MSA_employ2021 = MSA_employ2021[['AREA', 'TOT_EMP']] ### Area titles may be written differently, but the codes should be consistent overtime
MSA_employ2021 = MSA_employ2021.rename(columns={'TOT_EMP': 'TOT_EMP2021'})

MSA_employ2022 = create_MSAEmploy(Employment2022) 
MSA_employ2022 = MSA_employ2022[['AREA', 'TOT_EMP']]
MSA_employ2022 = MSA_employ2022.rename(columns={'TOT_EMP': 'TOT_EMP2022'})

MSA_employ2023 = create_MSAEmploy(Employment2023) 
MSA_employ2023 = MSA_employ2023[['AREA', 'TOT_EMP']]
MSA_employ2023 = MSA_employ2023.rename(columns={'TOT_EMP': 'TOT_EMP2023'})

In [33]:
# Pull the 2024 values defined in the map section
MSA_employ2024 = MSA_employ[['AREA', 'AREA_TITLE', 'TOT_EMP']]
MSA_employ2024 = MSA_employ2024.rename(columns={'TOT_EMP': 'TOT_EMP2024'})

## Create MSA Maps

The 2024 MSA definitions were slightly altered based on what was used in the 2021-2023 period. One notable metro area that was reassigned is the Boston area. 

For the sake of comparison, we will still look at growth rates, but this is notable limitation to this analysis. 
Since some metros don't map exactly anymore, we will just "clean" or map problematic metro codes if they have greater then 500 employees in the 2024 dataframe.

In [34]:
# 1) Put your dfs in a list
dfs = [MSA_employ2021, MSA_employ2022, MSA_employ2023]

# 2) Define a mapping old→new
area_map = {
    71650: 14460,  # Boston-Cambridge-Newton, MA-NH
    77200: 39300,  # Providence-Warwick, RI-MA
    17460: 17410,  # Cleveland, OH
    73450: 25540,  # Hartford-West Hartford-East Hartford, CT
    76750: 38860,  # Portland-South Portland, ME
}

# 3) Loop and replace in one go
for df in dfs:
    df['AREA'].replace(area_map, inplace=True)


In [35]:
MSA_employ_trend = pd.merge(MSA_employ2024, MSA_employ2021, left_on='AREA', right_on='AREA', how='left') ### only merging matches with 2024
MSA_employ_trend = pd.merge(MSA_employ_trend, MSA_employ2022, left_on='AREA', right_on='AREA', how='left')   
MSA_employ_trend = pd.merge(MSA_employ_trend, MSA_employ2023, left_on='AREA', right_on='AREA', how='left')  

new_order = ['AREA', 'AREA_TITLE', 'TOT_EMP2021', 'TOT_EMP2022', 'TOT_EMP2023','TOT_EMP2024']
MSA_employ_trend = MSA_employ_trend[new_order]


In [36]:
MSA_employ_trend['PercentGrowth_3yr'] = ((MSA_employ_trend['TOT_EMP2024'] - MSA_employ_trend['TOT_EMP2021']) / MSA_employ_trend['TOT_EMP2021']) * 100
MSA_employ_trend = MSA_employ_trend.sort_values('PercentGrowth_3yr', ascending=False) ### Sort by fastest growing

MSA_employ_trend['PercentGrowth_2yr'] = ((MSA_employ_trend['TOT_EMP2024'] - MSA_employ_trend['TOT_EMP2022']) / MSA_employ_trend['TOT_EMP2022']) * 100

MSA_employ_trend['PercentGrowth_1yr'] = ((MSA_employ_trend['TOT_EMP2024'] - MSA_employ_trend['TOT_EMP2023']) / MSA_employ_trend['TOT_EMP2023']) * 100


## Look at top 25 metros (based on 2024 employment counts)

In [40]:
top25 = MSA_employ_trend.nlargest(25, 'TOT_EMP2024')
top25 = top25.sort_values('PercentGrowth_3yr')
